In [2]:
import tabula
import numpy as np
import pandas as pd
import PyPDF2

In [3]:
file1 = "../doc/BMS.pdf"

In [13]:
registers_map = []
registers_bit = []

table_list = tabula.read_pdf(file1, pages='1-131')
for table in table_list:
    if "ADDRESS" in table.head() and "POR" in table.head() and "NAME" in table.head() and "DESCRIPTION" in table.head():
        registers_map.append(table.astype('string'))
    if "BIT" in table.head() and "POR" in table.head() and "NAME" in table.head() and "DESCRIPTION" in table.head():
        table = table.astype('string')
        table = table[table["NAME"] != "<NA>"]
        registers_bit.append(table)
registers_map = pd.concat(registers_map)
print(str(len(registers_map)) + " "+str(len(registers_bit)))

59 48


In [18]:
default_namespace = "MAX17843"
table_namespace = "registers_table"
namespace = f"{default_namespace}::{table_namespace}"
with open("MAX17843.h", "w") as BMS_lib:
    BMS_lib.write(f"namespace {namespace}")
    BMS_lib.write("{\n")
    for index, row in registers_map.iterrows():
        BMS_lib.write(f"#define {row['NAME']}_ADDR {row['ADDRESS']} //{row['DESCRIPTION']} address\n")
        
        if row["NAME"].find("CELL") == -1:
            table_namespace = row["NAME"]
            namespace = f"{default_namespace}::{table_namespace}"
            BMS_lib.write("{\n")
            for _, bit_row in registers_bit[index].iterrows():
                BMS_lib.write(f"#define {bit_row['NAME'].split('[')[0]}_BIT VERSION[{int(bit_row['BIT'].replace('D', '')) - int(bit_row['NAME'].split('[')[1].split(':')[0])}] //{bit_row['DESCRIPTION']}\n")
                BMS_lib.write(f"#define {bit_row['NAME'].split('[')[0]}_LENGTH {int(bit_row['NAME'].split('[')[1].split(':')[0])}")
                BMS_lib.write("\n")
            BMS_lib.write("}")
            BMS_lib.write("\n")
        else:
            index = index - 1
    BMS_lib.write("}\n")


IndexError: list index out of range

In [ ]:
table_list = tabula.read_pdf(file1,pages='100')
version_register = table_list[1].astype('string')
version_register = version_register[version_register["NAME"] != "<NA>"]
version_register

,BIT,POR,NAME,DESCRIPTION
0,D15,843h,MOD[11:0],Model number. Always reads 843h.
12,D3,1h,VER[3:0],Die version as below: MAX17843 = 1h


In [ ]:
table_namespace = "version"
namespace = f"{default_namespace}::{table_namespace}"
with open("MAX17843.h", "a") as BMS_lib:
    BMS_lib.write(f"namespace {namespace}")
    BMS_lib.write("{\n")
    for _, row in version_register.iterrows():
        BMS_lib.write(f"#define {row['NAME'].split('[')[0]}_BIT VERSION[{int(row['BIT'].replace('D', '')) - int(row['NAME'].split('[')[1].split(':')[0])}] //{row['DESCRIPTION']}\n")
        BMS_lib.write(f"#define {row['NAME'].split('[')[0]}_LENGTH {int(row['NAME'].split('[')[1].split(':')[0])}")
        BMS_lib.write("\n")
    BMS_lib.write("}")